<a href="https://colab.research.google.com/github/hyuna0926/cp2_phase2/blob/main/2%EC%9B%94%208%EC%9D%BC/als_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 46.9 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from scipy import sparse
from scipy.sparse import csr_matrix
from tqdm.notebook import tqdm
import numpy as np

from implicit.als import AlternatingLeastSquares as ALS
import implicit
import random
import os

/usr/local/lib/python3.8/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [5]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [6]:
trans = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/cart_purchase.parquet')
product = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/product.parquet')

In [79]:
product[['gender','masterCategory','subCategory','articleType','baseColour']] = product[['gender','masterCategory','subCategory','articleType','baseColour']].astype('str')
product['info'] = product['gender'] + ' ' + product['masterCategory'] + ' ' + product['subCategory'] + ' ' + product['articleType'] + ' ' + product['baseColour']
product

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,p_idx,info
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,27781,Men Apparel Topwear Shirts Navy Blue
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,20698,Men Apparel Bottomwear Jeans Blue
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,27546,Women Accessories Watches Watches Silver
3,59262,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,27546,Women Accessories Watches Watches Silver
4,59223,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,27546,Women Accessories Watches Watches Silver
...,...,...,...,...,...,...,...,...,...,...,...,...
44412,12544,Women,Apparel,Topwear,Tshirts,Peach,Fall,2011,Casual,Tantra Women Printed Peach T-shirt,27138,Women Apparel Topwear Tshirts Peach
44413,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013,Casual,Gas Men Caddy Casual Shoe,11124,Men Footwear Shoes Casual Shoes White
44414,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011,Casual,Lotto Men's Soccer Track Flip Flop,15896,Men Footwear Flip Flops Flip Flops Red
44415,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011,Casual,Puma Men Graphic Stellar Blue Tshirt,22174,Men Apparel Topwear Tshirts Blue


In [80]:
product = product[['p_idx','productDisplayName','info']]
#product.columns = ['iid', 'product_name', 'info']
product

,p_idx,productDisplayName,info
0,27781,Turtle Check Men Navy Blue Shirt,Men Apparel Topwear Shirts Navy Blue
1,20698,Peter England Men Party Blue Jeans,Men Apparel Bottomwear Jeans Blue
2,27546,Titan Women Silver Watch,Women Accessories Watches Watches Silver
3,27546,Titan Women Silver Watch,Women Accessories Watches Watches Silver
4,27546,Titan Women Silver Watch,Women Accessories Watches Watches Silver
...,...,...,...
44412,27138,Tantra Women Printed Peach T-shirt,Women Apparel Topwear Tshirts Peach
44413,11124,Gas Men Caddy Casual Shoe,Men Footwear Shoes Casual Shoes White
44414,15896,Lotto Men's Soccer Track Flip Flop,Men Footwear Flip Flops Flip Flops Red
44415,22174,Puma Men Graphic Stellar Blue Tshirt,Men Apparel Topwear Tshirts Blue


In [ ]:
trans 

,created_at,customer_id,product_id,p_idx,event,c_idx
0,2018-07-29 15:22:01,5868,54728,29317,purchase,3039
1,2021-01-15 19:52:40,82831,54728,29317,purchase,41968
2,2016-10-20 10:16:35,82831,34982,8394,purchase,41968
3,2018-08-26 14:09:52,82831,8627,9052,purchase,41968
4,2018-03-17 15:08:08,82831,46703,23994,purchase,41968
...,...,...,...,...,...,...
4958296,2022-06-25 16:21:40,31766,42861,10873,cart,16256
4958343,2019-03-18 07:59:22,21264,47192,10702,cart,10927
4958355,2021-05-05 21:17:51,23354,11872,10714,cart,12019
4958358,2019-09-22 16:10:57,99966,7775,6512,cart,50678


In [8]:
trans['interest'] = 1
trans_mf = trans[['c_idx','p_idx','interest']]
trans_mf

,c_idx,p_idx,interest
0,3039,29317,1
1,41968,29317,1
2,41968,8394,1
3,41968,9052,1
4,41968,23994,1
...,...,...,...
4958296,16256,10873,1
4958343,10927,10702,1
4958355,12019,10714,1
4958358,50678,6512,1


In [ ]:
trans['c_idx'].nunique()

50698

In [9]:
trans_als = trans_mf.groupby('c_idx')['interest'].sum()

In [10]:
trans_als

c_idx
0        152
1         14
2         16
3          4
4         21
        ... 
50693     29
50694     79
50695      4
50696     40
50697     12
Name: interest, Length: 50698, dtype: int64

In [11]:
customer_order = trans_mf.groupby('c_idx')['interest'].sum().to_frame().reset_index()
customer_order = customer_order.sort_values('interest', ascending=False)
customer_order[customer_order['interest'] > 24]
s = customer_order[customer_order['interest'] > 24].index.tolist()

In [12]:
trans_cb=trans.copy()
trans_cb = trans[~trans['c_idx'].isin(s)]
trans_cb

,created_at,customer_id,product_id,p_idx,event,c_idx,interest
0,2018-07-29 15:22:01,5868,54728,29317,purchase,3039,1
1586,2022-02-12 14:27:22,72919,54728,29317,purchase,37051,1
1587,2022-04-29 19:55:35,72919,48198,28371,purchase,37051,1
1588,2022-05-18 21:44:03,72919,24557,29615,purchase,37051,1
1589,2021-12-17 10:25:07,72919,36391,24657,purchase,37051,1
...,...,...,...,...,...,...,...
4958296,2022-06-25 16:21:40,31766,42861,10873,cart,16256,1
4958343,2019-03-18 07:59:22,21264,47192,10702,cart,10927,1
4958355,2021-05-05 21:17:51,23354,11872,10714,cart,12019,1
4958358,2019-09-22 16:10:57,99966,7775,6512,cart,50678,1


In [ ]:
trans_test['c_idx'].nunique()

31223

In [13]:
trans_als = trans_mf[trans_mf['c_idx'].isin(s)]
trans_als

,c_idx,p_idx,interest
1,41968,29317,1
2,41968,8394,1
3,41968,9052,1
4,41968,23994,1
5,41968,2497,1
...,...,...,...
4916676,48783,22111,1
4916678,48783,24342,1
4916680,48783,421,1
4916681,48783,10707,1


In [ ]:
trans_als['c_idx'].nunique()

19475

In [14]:
user_items = sparse.csr_matrix((trans_als['interest'],(trans_als['c_idx'], trans_als['p_idx'])))

In [15]:
matrix_size = user_items.shape[0]* user_items.shape[1]
num_purchases = len(user_items.nonzero()[0])
sparsity = 100 * (1 - (num_purchases / matrix_size))
sparsity

99.89795798828082

In [16]:
test = trans_als.groupby('c_idx').sample(frac=0.2,random_state=42)
train = trans_als.drop(test.index)

In [17]:
user_items = sparse.csr_matrix((train['interest'],(train['c_idx'], train['p_idx'])))

In [18]:
train_df = train.groupby('c_idx')['p_idx'].unique().to_frame().reset_index()
train_df

,c_idx,p_idx
0,0,"[27541, 6337, 8625, 3202, 9968, 6319, 3323, 21..."
1,6,"[3019, 3425, 24548, 21018, 23033, 4213, 25431,..."
2,7,"[16671, 15415, 20239, 1970, 29108, 18713, 2791..."
3,9,"[3513, 15950, 7362, 29679, 16425, 6040, 26990,..."
4,12,"[24805, 20834, 2489, 21592, 12300, 13786, 2012..."
...,...,...
19470,50689,"[15124, 30359, 11274, 24729, 18413, 22043, 256..."
19471,50690,"[25847, 1573, 5527, 24320, 5560, 9617, 14140, ..."
19472,50693,"[26461, 23942, 21796, 29029, 25744, 27675, 232..."
19473,50694,"[14136, 16954, 2578, 57, 20124, 3562, 6393, 12..."


In [19]:
test_df = test.groupby('c_idx')['p_idx'].unique().to_frame().reset_index()
test_df

,c_idx,p_idx
0,0,"[3393, 14555, 9702, 25009, 24439, 19617, 8865,..."
1,6,"[21480, 247, 18336, 16915, 30450, 24717, 251, ..."
2,7,"[8041, 23822, 28155, 22725, 12563, 11716, 2271..."
3,9,"[15742, 5355, 7848, 14545, 13754, 27263, 10766]"
4,12,"[17098, 21143, 9019, 15387, 6597, 5075, 26468,..."
...,...,...
19470,50689,"[20818, 29966, 14448, 24863, 4467, 12438, 2439..."
19471,50690,"[27765, 21300, 25342, 27911, 21037, 4991, 3853..."
19472,50693,"[886, 23955, 20891, 18967, 11382, 7842]"
19473,50694,"[15300, 11876, 8342, 571, 20799, 11976, 29680,..."


In [70]:
ALS = implicit.als.AlternatingLeastSquares(
    factors=200, regularization=40, alpha=40,
    iterations=20, random_state=42, calculate_training_loss=True
)

In [71]:
ALS.fit(user_items)

  0%|          | 0/20 [00:00<?, ?it/s]

In [72]:
user_vecs = ALS.user_factors
item_vecs = ALS.item_factors

In [35]:
product = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/product.parquet')

In [36]:
product_c =product.drop_duplicates(subset=['productDisplayName'], keep='first',ignore_index=True)

In [73]:
def recommendation(customer_idx,k):
  item = ALS.recommend(customer_idx,user_items[customer_idx],k)[0]
  recommend=pd.DataFrame(columns=product_c.columns)
  for i in item:
    item_re= product_c[product_c['p_idx'].isin([i])]
    recommend=pd.concat([recommend,item_re])
  return recommend

In [67]:
recommendations_t = ALS.recommend(0, user_items[0],filter_already_liked_items=True,N=25)

In [68]:
recommendations_t[0]

array([15841, 30254,  7934,  9002,  7938, 10707, 15287,  2580,  5248,
       24064, 27214, 28988, 11861, 11870, 12477,  4543, 24194, 24200,
        5355,  8526, 24190, 25024, 15286,  5260, 20035], dtype=int32)

In [74]:
def precision_recall(user_id):
  
  count=0
  user_buy = test[test['c_idx']==user_id].p_idx
  recommendations_t = ALS.recommend(user_id, user_items[user_id],filter_already_liked_items=True,N=25)
  for i in user_buy:
    for j in recommendations_t[0]:
      if i==j:
        count+=1
  
  precision = count/len(recommendations_t[0])

  try: # userbuy가 없는 데이터셋이 존재
    recall = count/len(user_buy)
    return precision, recall

  except:
    pass
  

In [28]:
recommendation(0,25)

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,p_idx
5062,39638,Men,Footwear,Shoes,Sports Shoes,White,Summer,2012,Sports,Lotto Men White Sports Shoes,15841
13367,34874,Women,Accessories,Bags,Handbags,Blue,Summer,2012,Casual,Envirosax Women Blue Bag,7934
3080,47190,Men,Footwear,Shoes,Formal Shoes,Brown,Summer,2012,Formal,Franco Leone Men Brown Shoes,10707
329,50741,Men,Footwear,Shoes,Casual Shoes,Black,Winter,2018,Casual,Red Chief Men Black Shoes,24064
6333,25202,Women,Accessories,Bags,Handbags,Beige,Winter,2015,Casual,Lino Perros Women Leatherette Beige Handbag,15286
12743,25280,Women,Accessories,Bags,Handbags,Black,Summer,2012,Casual,Lino Perros Women Leatherette Black Handbag,15287
2149,14805,Women,Accessories,Bags,Handbags,Black,Summer,2011,Casual,United Colors of Benetton Women Solid Black Ha...,28988
785,23249,Men,Footwear,Shoes,Formal Shoes,Black,Fall,2011,Formal,Arrow Men Formal Black Shoes,2632
13917,46801,Men,Footwear,Shoes,Sports Shoes,Grey,Summer,2012,Sports,Nike Men Grey & White Sports Shoes,18545
155,24250,Men,Footwear,Shoes,Casual Shoes,White,Fall,2012,Casual,Numero Uno Men White Casual Shoes,20035


In [48]:
precision_recall(50689)

(0.0, 0.0)

In [69]:
len(test_df['c_idx'].values)

19475

In [75]:
precision_list = []
recall_list = []


In [76]:
for user in test_df['c_idx'].values :
  precision, recall = precision_recall(user)
  precision_list.append(precision)
  recall_list.append(recall)


In [77]:
precision_at_k=np.mean(precision_list)
recall_at_k=np.mean(recall_list)

In [78]:
precision_at_k , recall_at_k

(0.007379717586649551, 0.01107586949557767)